# Actividad 1: Métricas de regresión

Vamos a comenzar cargando el .csv proporcionado como material de la práctica y definiendo una serie de funciones que nos permitirán calcular todas las métricas de regresión estudiadas en sesiones de teoría.

In [1]:
import pandas as pd
import numpy as np

df_1 = pd.read_csv('l2p1.csv')
df_1.head()

,Y objetivo,Predicciones M1,Predicciones M2
0,2.50,3.0,2.0
1,3.00,2.9,2.0
2,1.60,2.0,2.0
3,8.00,8.1,7.0
4,4.56,4.0,5.0


Vemos que el fichero está compuesto por tres columnas:

- ``Y objetivo``: Valores reales de la variable contínua a predecir.
- ``Predicciones M1``: Valores predichos por el modelo M1.
- ``Predicciones M2``: Valores predichos por el modelo M2.

Calcularemos las métricas de regresión sobre los valores de las predicciones. Pasamos a definir las funciones de cálculo de dichas magnitudes. Por cuestiones de eficiencia, utilizaremos arrays ``numpy`` y sus funciones predefinidas. 

In [2]:
def mean_absolute_error(x_real, x_pred):
    return (1 / len(x_real)) * np.sum(np.abs(x_real - x_pred))

def mean_squared_error(x_real, x_pred):
    return (1 / len(x_real)) * np.sum(np.square(x_real - x_pred))

def root_mean_squared_error(x_real, x_pred):
    return np.sqrt((1 / len(x_real)) * np.sum(np.square(x_real - x_pred)))

def mean(x):
    return np.sum(x) / len(x)

def variance(x):
    return np.sqrt(np.sum(np.square(x - mean(x))))    

def covariance(x1, x2):
    return np.sum((x1 - mean(x1)) * (x2 - mean(x2)))

def coef_det(x_real, x_pred):
    return covariance(x_real, x_pred) / (variance(x_real) * variance(x_pred))  

Pasamos a continuación a obtener las métricas para cada modelo.

In [3]:
y_objetivo = np.array(df_1['Y objetivo'].values)
pred_m1 = np.array(df_1['Predicciones M1'].values)
pred_m2 = np.array(df_1['Predicciones M2'].values)

In [4]:
# Obtenemos las métricas para M1
print(f'Error medio absoluto de M1 {mean_absolute_error(y_objetivo, pred_m1)}')
print(f'Error cuadrático medio de M1 {mean_squared_error(y_objetivo, pred_m1)}')
print(f'Raíz del error cuadrático medio de M1 {root_mean_squared_error(y_objetivo, pred_m1)}')
print(f'Coeficiente de determinación de M1 {coef_det(y_objetivo, pred_m1)}')

Error medio absoluto de M1 1.0396666666666667
Error cuadrático medio de M1 7.983049999999999
Raíz del error cuadrático medio de M1 2.8254291709402306
Coeficiente de determinación de M1 0.7925094612153079


In [5]:
# Obtenemos las métricas para M2
print(f'Error medio absoluto de M2 {mean_absolute_error(y_objetivo, pred_m2)}')
print(f'Error cuadrático medio de M2 {mean_squared_error(y_objetivo, pred_m2)}')
print(f'Raíz del error cuadrático medio de M2 {root_mean_squared_error(y_objetivo, pred_m2)}')
print(f'Coeficiente de determinación de M2 {coef_det(y_objetivo, pred_m2)}')

Error medio absoluto de M2 0.6113333333333333
Error cuadrático medio de M2 0.5219533333333333
Raíz del error cuadrático medio de M2 0.7224633785413163
Coeficiente de determinación de M2 0.9892110230257817


Viendo los resultados obtenidos parece una mejor elección elegir el modelo M2 en lugar del M1. Esto es así porque, por una parte, los errores medios tienen un valor más bajo con M2 mientras que el coeficiente de correlación presenta un valor más cercano a 1 con las predicciones de M2. 

# Actividad 2: Métricas de clasificación binaria

De nuevo, comenzamos cargando el .csv proporcionado como material de la práctica y definiendo las funciones necesarias para calcular la matriz de confusión, así como todas las métricas de clasificación solicitadas.

In [6]:
df_2 = pd.read_csv('l2p2.csv')
df_2.head()

,Clase Objetivo,Predicciones M1,Predicciones M2
0,0,1,0
1,0,0,0
2,0,0,0
3,1,1,1
4,1,1,1


La composición de este fichero es similar a la estudiada en el ejercicio anterior. La única diferencia reside en el hecho de que, al estar estudiando una clasificación binaria, las clases se denotan como 0 ó 1. 

In [7]:
def confusion_matrix(x_real, x_pred):
    # Vamos a hacer una definición general de esta función, es decir, 
    # se podrá utilizar en problemas de clasificación en dos o más clases.
    # Obtenemos las diferentes clases del problema
    classes = np.unique(x_real)
    n = len(classes)
    # Definimos la matriz inicialmente toda a 0s
    matrix = np.zeros(shape = (n, n))
    # Comenzamos a rellenar la matriz
    for i in range(n):
        # Obtenemos la clase actual
        current_class = classes[i]
        # Posiciones donde el valor real vale current_class
        pos1 = np.where(x_real == current_class)
        for j in range(n):
            # Obtenemos las posiciones de la classes[j]
            pos2 = np.where(x_pred == classes[j])
            # Contamos los aciertos/fallos viendo en cuantas posiciones
            # coinciden pos1 y pos2.
            # Si classes[j] == current_class las posiciones donde coinciden
            # pos1 y pos2 son los aciertos
            # Si classes[j] != currenc_class las posiciones donde coinciden
            # pos1 y pos2 son los fallos del modelo
            matrix[i, j] = len(np.intersect1d(pos1, pos2))
    return matrix
            
def true_values(matrix):
    # Los valores correctamente clasificados se encuentran en la diagonal
    # de la matriz de confusión
    return np.diagonal(matrix)    
    
def accuracy(matrix):
    return np.sum(true_values(matrix)) / np.sum(matrix)

def recall(matrix):
    sum_rows = np.sum(matrix, axis = 1)
    return true_values(matrix) / sum_rows

def fp_rate(matrix):
    sum_columns = np.sum(matrix, axis = 0)
    fp = sum_columns - true_values(matrix)
    tn = np.zeros((1, len(matrix))).flatten()
    for i in range(len(matrix)):
        tn[i] = np.sum(np.delete(np.delete(matrix, i, axis = 0), i, axis = 1))
    return fp / (tn + fp)
    
    #sum_columns = np.sum(matrix, axis = 0)
    #sum_rows = np.zeros((1, len(matrix)))
    #for i in range(len(sum_rows)):
    #    sum_rows[i] = np.sum(np.delete(matrix, i, axis = 0))
    #return (sum_columns - true_values(matrix)) / sum_rows.flatten()

def specificity(matrix):
    sum_columns = np.sum(matrix, axis = 0)
    fp = sum_columns - true_values(matrix)
    tn = np.zeros((1, len(matrix))).flatten()
    for i in range(len(matrix)):
        tn[i] = np.sum(np.delete(np.delete(matrix, i, axis = 0), i, axis = 1))
    return tn / (tn + fp)
    
    
    #true_val = true_values(matrix)
    #aux_true = np.zeros((1, len(true_val)))
    #sum_rows = np.zeros((1, len(matrix)))
    #for i in range(len(aux_true)):
    #    aux_true[i] = np.sum(np.delete(true_val, i))
    #    sum_rows[i] = np.sum(np.delete(matrix, i, axis = 0))
    #return aux_true.flatten() / sum_rows.flatten()

def precission(matrix):
    sum_columns = np.sum(matrix, axis = 0)
    return true_values(matrix) / sum_columns

def f1_score(matrix):
    prec = precission(matrix)
    rec = recall(matrix)
    return (2 * prec * rec) / (prec + rec)

def kappa(matrix):
    p0 = accuracy(matrix)
    sum_rows = np.sum(matrix, axis = 1)
    sum_columns = np.sum(matrix, axis = 0)
    pe = (1 / (np.sum(matrix) ** 2)) * np.sum(sum_rows * sum_columns)
    return (p0 - pe) / (1 - pe)

Obtenemos la matriz de confusión de cada modelo

In [8]:
y_objetivo = np.array(df_2['Clase Objetivo'].values)
pred_m1 = np.array(df_2['Predicciones M1'].values)
pred_m2 = np.array(df_2['Predicciones M2'].values)

In [9]:
conf_m1 = confusion_matrix(y_objetivo, pred_m1)
conf_m1

array([[19.,  1.],
       [ 4.,  6.]])

Para las predicciones de M1 vemos la siguiente distribución de aciertos/fallos:

- Verdaderos positivos: 19 elementos clasificados correctamente como 0.
- Verdaderos negativos: 6 elementos clasificados correctamente como 1.
- Falsos positivos: 1 elemento clasificado incorrectamente como 0.
- Falsos negativos: 4 elementos clasificados incorrectamente como 1.

Sumando la diagonal podemos observar como se obtienten 25 clasificaciones correctas de 30 patrones a clasificar. Vamos a obtener las métricas de esta clasificación.

In [10]:
print(f'CCR M1 -> {accuracy(conf_m1)}')
print(f'Recall M1 -> {recall(conf_m1)[0]}')
print(f'FP Rate M1 -> {fp_rate(conf_m1)[0]}')
print(f'Especifidad M1 -> {specificity(conf_m1)[0]}')
print(f'Precisión M1 -> {precission(conf_m1)[0]}')
print(f'F1-Score M1 -> {f1_score(conf_m1)[0]}')
print(f'Kappa M1 -> {kappa(conf_m1)}')

CCR M1 -> 0.8333333333333334
Recall M1 -> 0.95
FP Rate M1 -> 0.4
Especifidad M1 -> 0.6
Precisión M1 -> 0.8260869565217391
F1-Score M1 -> 0.8837209302325583
Kappa M1 -> 0.5945945945945946


In [11]:
conf_m2 = confusion_matrix(y_objetivo, pred_m2)
conf_m2

array([[12.,  8.],
       [ 1.,  9.]])

Para las predicciones de M2 vemos la siguiente distribución de aciertos/fallos:

- Verdaderos positivos: 12 elementos clasificados correctamente como 0.
- Verdaderos negativos: 9 elementos clasificados correctamente como 1.
- Falsos positivos: 8 elemento clasificado incorrectamente como 0.
- Falsos negativos: 1 elementos clasificados incorrectamente como 1.

Sumando la diagonal podemos observar como se obtienten 21 clasificaciones correctas de 30 patrones a clasificar. Vamos a obtener las métricas de esta clasificación.

In [12]:
print(f'CCR M2 -> {accuracy(conf_m2)}')
print(f'Recall M2 -> {recall(conf_m2)[0]}')
print(f'FP Rate M2 -> {fp_rate(conf_m2)[0]}')
print(f'Especifidad M2 -> {specificity(conf_m2)[0]}')
print(f'Precisión M2 -> {precission(conf_m2)[0]}')
print(f'F1-Score M2 -> {f1_score(conf_m2)[0]}')
print(f'Kappa M2 -> {kappa(conf_m2)}')

CCR M2 -> 0.7
Recall M2 -> 0.6
FP Rate M2 -> 0.1
Especifidad M2 -> 0.9
Precisión M2 -> 0.9230769230769231
F1-Score M2 -> 0.7272727272727274
Kappa M2 -> 0.42553191489361697


Viendo los resultados obtenidos, el modelo M1 parece obtener valores más favorables en todas las métricas de clasificación. Obtiene una precisión más alta, un recall más alto y por tanto, un valor F1 más alto, lo que comparativamente hace que el modelo M1 sea más favorable en este caso particular.

# Cuestión 3: Clasificación y métricas en tres clases

Vamos a construir los resultados propuestos en el manual de la actividad como un DataFrame.

In [13]:
df_3 = pd.DataFrame({"Gato": [20, 5, 5], "Perro": [10, 30, 5], "Loro": [5, 0, 25]}, 
                   index = ["Gato", "Perro", "Loro"])
df_3

,Gato,Perro,Loro
Gato,20,10,5
Perro,5,30,0
Loro,5,5,25


Construimos otro DataFrame donde mostramos las métricas de regresión para cada clase.

In [14]:
columns = ["CCR", "Recall", "FP Rate", "Especifidad", "Precisión", "F1-Score", "Kappa"]
rows = ["Gato", "Perro", "Loro"]
matrix_df = np.array(df_3.values)
ccr = accuracy(matrix_df)
rec = recall(matrix_df)
fp_r = fp_rate(matrix_df)
esp = specificity(matrix_df)
prec = precission(matrix_df)
f1_s = f1_score(matrix_df)
kappas = kappa(matrix_df)
aux = [ccr, rec, fp_r, esp, prec, f1_s, kappas]
df_res = pd.DataFrame({x:y for (x, y) in zip(columns, aux)}, index = rows)
df_res

,CCR,Recall,FP Rate,Especifidad,Precisión,F1-Score,Kappa
Gato,0.714286,0.571429,0.142857,0.857143,0.666667,0.615385,0.571429
Perro,0.714286,0.857143,0.214286,0.785714,0.666667,0.750000,0.571429
Loro,0.714286,0.714286,0.071429,0.928571,0.833333,0.769231,0.571429


El valor CCR es igual en todas las filas pues es un valor de precisión global que, en este caso, alcanza el 71.4285% de valores predichos correctamente.